In [ ]:
import random
from tqdm import tqdm
from yaml import safe_load
import json
random.seed(42)
from opendet2.data.voc_coco import VOC_COCO_CATEGORIES
import numpy as np
from pathlib import Path

file_stem = '/media/prakash/LaCie1/Backup/opendet2/8gpu_outputs/faster_rcnn_R_50_FPN_3x_opendet_run1_aw_0.017_lw_0.21_best_model/pascal_voc_eval/'
logits = []
labels = []
######
### Read Logits from the assiciated data
### This means the logits of only the detections are displayed.
#######
# Read the JSON file
tmp_arr = []
tmp_label = []
tmp_name = []
concerned_ids =  np.arange(20).tolist() + [80]
TOTAL_CLASSES=VOC_COCO_CATEGORIES
for id in range (len(TOTAL_CLASSES)):
    with open(file_stem + TOTAL_CLASSES[id]+'embeddings.json' , 'r') as file:
        data = json.load(file)
    # Collate all the array lists into one NumPy array
    for cnt,key in enumerate(data):
        if id==80:
            max_det = 1000
        else:
            max_det=300
        if (id in concerned_ids and cnt<max_det ): #and int(key)<1050):
            tmp_arr.append(np.array(data[key]))
            #tmp_label.append(id)     
            tmp_label.append(id) 

total_feat = np.array(tmp_arr)#[1000:-1,:]
labels = np.array(tmp_label)#[100:-1]
print (total_feat.shape)
print (np.unique(labels))


In [ ]:
#######
#### UMAP
#######
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import umap
import random
from matplotlib.colors import ListedColormap
random.seed(42)
plt.figure()
clusterable_embedding = umap.UMAP(
    n_neighbors=400,
    min_dist=1,
    n_components=3,
    random_state=40,
).fit_transform(total_feat )

#clusterable_embedding = umap.UMAP(n_components=3,min_dist=1,n_neighbors=300,random_state=42).fit_transform(total_feat )
generic_fontsize = 16

'''rev_names = {}
for k,v in classes.items():
    v=v-1
    rev_names[k] = v '''

label_colors = {0: '#FF0000', 1: '#00FF00', 2: '#0000FF', 3: '#FFFF00', 4: '#FF00FF',
                5: '#00FFFF', 6: '#FFA500', 7: '#800080', 8: '#FFA500' , 9: '#008080',
                10: '#800000', 11: '#808000', 12: '#1f77b4', 13: '#00FF80', 14: '#008000' ,
                15: '#ff7f0e', 16: '#808080', 17: '#80FFFF', 18: '#0080FF', 19: '#8000FF',
                20: '#FF8000', 21: '#008080', 22: '#80FF00', 23: '#FF8080', 24: '#80FFFF',
                80: '#000000'}


# Create a scatter plot
for label in set(labels):
    #if label  not in [80]:
        #print (label_colors[label])
        mask = [l == label for l in labels]
        scatter = plt.scatter(np.array(clusterable_embedding[:, 0])[mask], np.array(clusterable_embedding[:, 1])[mask], label=label, color=label_colors[label])
        plt.legend(*scatter.legend_elements(),fontsize = 15)
#scatter = plt.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1], c=np.array(labels) , cmap=COLORMAP)
#plt.title(rev_names)

plt.xticks( fontsize=generic_fontsize, weight='bold')
plt.yticks(fontsize=generic_fontsize, weight='bold')
#plt.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1],def open_url(hoverData):
if clusterable_embedding.shape[1]==3:
    fig = plt.figure(figsize=(17, 8))
    ax = fig.add_subplot(111, projection='3d')
    for label in set(labels):
        #if label  not  in [80]:
            mask = [l == label for l in labels]
            if label == 80:
                mark = 'x'
            else:
                mark = 'o'
            scatter = ax.scatter(np.array(clusterable_embedding[:, 0])[mask], np.array(clusterable_embedding[:, 1])[mask], np.array(clusterable_embedding[:, 2])[mask],
                    label=label, c=label_colors[label], marker=mark)

    ax.legend(*scatter.legend_elements(),fontsize = 15)
    #plt.title(rev_names)

    #plt.xticks( fontsize=generic_fontsize, weight='bold')
    #plt.yticks(fontsize=generic_fontsize, weight='bold')
    ax.tick_params(axis='both', which='major', labelsize=12, width=3, length=6, pad=6)
    ax.tick_params(axis='both', which='minor', labelsize=10, width=2, length=4, pad=6)
    # Set z-axis label
    #ax.set_zlabel('Third Dimension',fontsize=generic_fontsize)
    ax.view_init(elev=13,azim=0)
    ax.legend()
    plt.title('Logit Space Visualisation of Detections on Holdout test set',fontsize = generic_fontsize)
    # Set legend font properties
    #legend = plt.legend(fontsize=30)
    # Get the legend handles and labels
    #handles, labels = plt.gca().get_legend_handles_labels()
    # Increase font size of x-axis go
    #plt.xlabel('First Dimension', fontsize=generic_fontsize)
    #plt.ylabel('Second Dimension', fontsize=generic_fontsize)
    #plt.zlabel('Third Dimension', fontsize=generic_fontsize)
    plt.savefig('3d.pdf',dpi=150)
    plt.show()

In [ ]:
import numpy as np

# Sample data: Replace this with your actual data
embeddings = total_feat

# Calculate the means of each class
class_means = {}
centroid = []
for label in np.unique(labels):
    #print (embeddings[labels == label].shape)
    centroid.append(np.mean(embeddings[labels == label], axis=0))
    class_means[label] =  np.mean(embeddings[labels == label], axis=0)
#print (class_means )
# Calculate the intraclass variance

intraclass_variance = 0.0

intra_dist_mean = []
intra_dist_var = []
for label in np.arange(20):
    
    #tmp_det = np.array(embeddings[labels == label]).shape [0]
    #print (class_means[label])
    #print (embeddings[labels == label])
    #print (np.linalg.norm(embeddings[labels == label] - class_means[label])**2)
    #print (label)

    #det +=tmp_det
    intra_dist_mean.append(np.mean(np.linalg.norm(embeddings[labels == label] - class_means[label], axis=1) ))
    intra_dist_var.append(np.var(np.linalg.norm(embeddings[labels == label] - class_means[label], axis=1)  ))
    #intraclass_variance +=  np.linalg.norm(embeddings[labels == label] - class_means[label])**2
#print (intra_dist_mean)
# Calculate the mean distance
intra_mean_distance = np.mean(intra_dist_mean)

# Calculate the variance of distances
intra_variance = np.mean(intra_dist_var)

#print("Mean Distance:", intra_mean_distance)
print ('mean of intra distance', intra_mean_distance)
print("Variance of Distances:", intra_variance)

# Calculate pairwise distances between class means

# Calculate pairwise distances between class means
distances = []
for i in range(len(class_means) - 1):
    for j in range(i + 1, len(class_means)):
        dist = np.linalg.norm(class_means[i] - class_means[j])  # Euclidean distance
        distances.append(dist)

# Calculate the mean distance
mean_distance = np.mean(distances)

# Calculate the variance of distances
#variance = np.var(distances)

print("Mean of inter distances :", np.mean(distances))
print("variance of inter Distances:", np.var(distances))
print (distances)


In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

def my_dunn_index_optimized(cluster_centers, cluster_labels, data):
    unique_labels = np.unique(cluster_labels)
    n_clusters = len(unique_labels)
    
    # Calculate diameters of clusters
    diameters = []
    for k in range(n_clusters):
        cluster_points = data[cluster_labels == k]
        if len(cluster_points) > 1:
            diameters.append(np.max(cdist(cluster_points, cluster_points)))
        else:
            diameters.append(0.0)
    
    max_diameter = max(diameters)

    # Calculate min distance between points in different clusters
    min_distances = []
    for i in range(n_clusters):
        for j in range(i + 1, n_clusters):
            points_i = data[cluster_labels == i]
            points_j = data[cluster_labels == j]
            if len(points_i) > 0 and len(points_j) > 0:
                min_distances.append(np.min(cdist(points_i, points_j)))
    
    min_distance = min(min_distances) if min_distances else 0.0
    print (min_distance)
    print (max_diameter)
    dunn_index = min_distance / max_diameter if max_diameter > 0 else 0.0

    return dunn_index

my_dunn_index_optimized(class_means, labels,embeddings)

In [ ]:
from sklearn.metrics import calinski_harabasz_score
import numpy as np

calinski_harabasz_index = calinski_harabasz_score(embeddings, labels)

print("Calinski-Harabasz Index:", calinski_harabasz_index)


In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

def hubert_index(data, labels):
    unique_labels = np.unique(labels)
    m = len(unique_labels)

    # Calculate the mean distance within clusters
    mean_within_cluster_distance = sum(
        np.mean(cdist(data[labels == label], data[labels == label]))
        for label in unique_labels if np.sum(labels == label) > 1
    ) / m

    # Calculate the mean distance between clusters
    mean_between_cluster_distance = sum(
        np.mean(cdist(data[labels == i], data[labels == j]))
        for i in range(m)
        for j in range(i + 1, m)
    ) / (m * (m - 1) / 2)

    # Calculate the Hubert Index
    hubert_index_value = mean_within_cluster_distance / mean_between_cluster_distance

    return hubert_index_value

 

hubert_index_value = hubert_index(embeddings, labels)
print("Hubert Index:", hubert_index_value)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import make_blobs


# Perform Fisher's LDA
lda = LinearDiscriminantAnalysis(n_components=3)
X_lda = lda.fit_transform(embeddings, labels)

# Analyze separation between clusters and intra-cluster properties
print("Fisher's LDA Separation Metric:", lda.score(embeddings, labels))



In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

def xie_beni_index(data,labels,centroids):
    # Fit KMeans clustering model
    k = len(np.unique(labels) )
    kmeans = KMeans(n_clusters= k)


    # Calculate the within-cluster dispersion
    dist_within_cluster = np.sum([np.sum((data[labels == i] - centroids[i])**2) for i in range(k)])

    # Calculate the distance from each point to the nearest cluster center
    dist_to_nearest_center = pairwise_distances_argmin_min(data, centroids)[1]

    # Calculate the Xie-Beni Index
    xie_beni_index_value = dist_within_cluster / (k * np.sum(dist_to_nearest_center))

    return xie_beni_index_value

xie_beni_index_value = xie_beni_index(embeddings, labels,centroid)
print("Xie-Beni Index:", xie_beni_index_value)


In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from scipy.stats import entropy
import seaborn as sns
import pickle

def calculate_free_energy(embeddings, kde_in, kde_out):
    log_dens_in = kde_in.score_samples(embeddings)
    log_dens_out = kde_out.score_samples(embeddings)

    free_energy_scores = -log_dens_in + log_dens_out
    return free_energy_scores

# Convert NumPy array to PyTorch tensor
embeddings_tr = torch.from_numpy(embeddings)
def plot_pdf(ax, data, label, color):
    sns.histplot(data, kde=True, ax=ax, label=label, color=color)
    ax.set_title('Free Energy PDF')
    ax.set_xlabel('Free Energy')
    ax.legend()
# Create KDE models for in-distribution and out-of-distribution embeddings
in_distribution_embeddings = total_feat[(labels >= 0)  & (labels <20)]  #np.random.multivariate_normal(mean=[0, 0], cov=[[1, 0.5], [0.5, 1]], size=1000)
out_distribution_embeddings =  total_feat[labels == 80]   #np.random.multivariate_normal(mean=[3, 3], cov=[[1, -0.5], [-0.5, 1]], size=1000)


kde_in = KernelDensity(bandwidth=0.5, kernel='gaussian')
kde_in.fit(in_distribution_embeddings)

kde_out = KernelDensity(bandwidth=0.5, kernel='gaussian')
kde_out.fit(out_distribution_embeddings)

# Calculate Free Energy scores for both in-distribution and out-of-distribution embeddings
free_energy_in = calculate_free_energy(in_distribution_embeddings, kde_in, kde_out)
free_energy_out = calculate_free_energy(out_distribution_embeddings, kde_in, kde_out)

fig, ax = plt.subplots(figsize=(10, 6))
plot_pdf(ax, free_energy_in, 'In-Distribution', 'blue')
plot_pdf(ax, free_energy_out, 'Out-of-Distribution', 'orange')

plt.show()

In [ ]:

#### Using the tSNE for visualization
import seaborn as sns
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 9))
# Apply t-SNE to reduce the dimensionality of the feature space
tsne = TSNE(n_components=3, random_state=1)  #TSNE(n_components=3,  perplexity=10, learning_rate=20)
#TSNE(n_components=3, random_state=0) 
embedded = tsne.fit_transform(total_feat)
import pandas as pd
tsne_df = pd.DataFrame({'X':embedded[:,0],
                        'Y':embedded[:,1]})
tsne_df['label'] = labels
# Visualize the t-SNE embedding
#sns.scatterplot(x="X", y="Y",
#              data=tsne_df, palette=COLORMAP)
#
# Create a scatter plot
for label in set(labels):
    #if label  not in [0,1,2,3,4]:
        #print (label_colors[label])
        mask = [l == label for l in labels]
        scatter = plt.scatter(np.array(embedded[:, 0])[mask], np.array(embedded[:, 1])[mask], label=label, color=label_colors[label])
        plt.legend(*scatter.legend_elements(),fontsize = 15)
#scatter = plt.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1], c=np.array(labels) , cmap=COLORMAP)
#plt.title(rev_names)
plt.xticks( fontsize=generic_fontsize, weight='bold')
plt.yticks(fontsize=generic_fontsize, weight='bold')
#plt.scatter(clusterable_embedding[:, 0], clusterable_embedding[:, 1],def open_url(hoverData):

if embedded.shape[1]==3:
    fig = plt.figure(figsize=(17, 8))
    ax = fig.add_subplot(111, projection='3d')
    for label in set(labels):
        mask = [l == label for l in labels]
        scatter = ax.scatter(np.array(embedded[:, 0])[mask], np.array(embedded[:, 1])[mask], np.array(embedded[:, ])[mask],
                    label=label, c=label_colors[label], marker='o')

    ax.legend(*scatter.legend_elements(),fontsize = 15)
    #plt.title(rev_names)
    #ax.scatter(tsne_df['X'], tsne_df['Y'],embedded[:,2], c=np.array(labels) , cmap=COLORMAP)
    ax.legend(*scatter.legend_elements())
    
    ax.set_xlabel('Dimension 1')
    ax.set_ylabel('Dimension 2')
    ax.set_zlabel('Dimension 3')
    #ax.legend()
    plt.show()